# Final Project NLP Smart Scientific Content Auto-completion System


a text auto-complete system that supports scientists in writing articles in:
- completing some typed words/letters.
- Identifying words/phrases that are very unlikely to occur in some text contexts and suggesting proper replacements.


Karmel Salah & Fatimah Najwan
15/4/2022

# We will use Colab for a Free GPU so if you are using Colab frist connect to your drive to get the dataset


In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#unzip the dataset folder
!unzip ./drive/MyDrive/dataset.zip 

Archive:  ./drive/MyDrive/dataset.zip
   creating: dataset/
  inflating: dataset/1012.2609.txt   
  inflating: dataset/1207.1847v1.txt  
  inflating: dataset/1209.3126v1.txt  
  inflating: dataset/1210.0852.txt   
  inflating: dataset/1305.6143.txt   
  inflating: dataset/1306.6802v2.txt  
  inflating: dataset/1308.0850v5.txt  
  inflating: dataset/1309.4168v1.txt  
  inflating: dataset/1312.3005v3.txt  
  inflating: dataset/1403.6652v2.txt  
  inflating: dataset/1404.3377v1.txt  
  inflating: dataset/1404.4326v1.txt  
  inflating: dataset/1405.4053v2.txt  
  inflating: dataset/1405.4273v1.txt  
  inflating: dataset/1406.1078v3.txt  
  inflating: dataset/1408.5882v2.txt  
  inflating: dataset/1409.0473v7.txt  
  inflating: dataset/1409.2329v5.txt  
  inflating: dataset/1409.3215v3.txt  
  inflating: dataset/1410.5329v4.txt  
  inflating: dataset/1410.8206v4.txt  
  inflating: dataset/1411.4555v2.txt  
  inflating: dataset/1412.1058v2.txt  
  inflating: dataset/1412.2007v2.txt  
  infla

# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import os, glob
import math
import matplotlib.pyplot as plt
import seaborn as sns
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

[nltk_data] Downloading package punkt to /home/karmel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/karmel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/karmel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/karmel/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-05-11 15:25:58.878825: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such fil

In [3]:
# get the names of each folders
files_names = os.listdir('./dataset')
len(files_names)

1064

# Convert to DataFrame

In [4]:
df = pd.DataFrame([], columns = ["id", "articles"])
print(f"data frame is {df}")

data frame is Empty DataFrame
Columns: [id, articles]
Index: []


In [5]:
def convert_to_df(df, files_names):
        id = []
        articles= []
        # Define files place and read each sub file for cleaning
        for index, file in enumerate(files_names):
                # read each article files
                path = os.getcwd()
                file_path = '{}/dataset/{}'.format(path,file)
                with open(file_path,'r', encoding='utf-16',  errors="ignore") as f:
                    texts = f.read() 
                    articles.append(texts) 
                    id.append(index)
                    f.close()
        print("the ids", id)
        print("the articles", articles)
        df['id'] = id
        df["articles"] = articles
        return df


In [6]:
df = convert_to_df(df, files_names)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
df["articles"]

0       Statistical Machine Translation\n\nDraft of Ch...
1       Found in Translation:\n\nLearning Robust Joint...
2       Available online at www.sciencedirect.com\n\nS...
3       J OURNAL OF I NFORMATION S CIENCE AND E NGINEE...
4       The University of Sheﬃeld\n\nT. E. Dunning\n\n...
                              ...                        
1059    Multi-label Hate Speech and Abusive Language D...
1060    hULMonA (      ): The Universal Language Model...
1061    Mazajak: An Online Arabic Sentiment Analyser\n...
1062    2017 2nd International Conferences on Informat...
1063    INVESTIGATIVE RADIOLOGY Volume 28, Number 6, 4...
Name: articles, Length: 1064, dtype: object

In [8]:
limited_df = pd.DataFrame(df['articles'].iloc[0:50])

In [9]:
# count the number of words for each doc
limited_df['word_count'] = limited_df['articles'].apply(lambda x: len(str(x).split()))

 # Text pre-processing:

In [10]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower()
    # Remove leading and trailing spaces
    text= text.strip()  
    text= re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text= re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text)
    # get ride of one letter word!
    text = re.sub(r'\b\w\b','',text) 
    
      
    return text

In [11]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/karmel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

In [13]:
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [14]:
import enchant

In [15]:
#Remove non English words
dictionary = enchant.Dict("en_US")
   
def remove_non_english(text):
        en_words= []
        for word in text:
            if dictionary.check(word):
                en_words.append(word)
        text = ' '.join(str(e) for e in en_words)
        return text

In [16]:
limited_df['english_text'] =limited_df['articles'].apply(lambda text: remove_non_english(text.split()))

In [17]:
#limited_df['english_text'][0]

In [18]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

In [19]:
limited_df['clean_text'] = limited_df['english_text'].apply(lambda x: finalpreprocess(x))


In [20]:
articles = limited_df['clean_text']

In [21]:
#Word2Vec
# Word2Vec runs on tokenized sentences
tok = [nltk.word_tokenize(i) for i in articles]  

In [22]:
#tok[:2]

In [23]:
model = Word2Vec(tok, min_count=1)

In [24]:
#vector = model.wv['computer']  # get numpy vector of a word
#sims = model.wv.most_similar('computer', topn=10)  # get other similar words

In [25]:
# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

In [26]:
#word_vectors.vectors

In [27]:
#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
        
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    


In [28]:
w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))
modelw = MeanEmbeddingVectorizer(w2v)

# converting texts to numerical data using Word2Vec
vectors_w2v = modelw.transform(tok)

In [29]:
#vectors_w2v

In [30]:
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models.coherencemodel import CoherenceModel
#vis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [31]:
id2word = corpora.Dictionary(tok)

corpus = []
for text in tok:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])
# Create Dictionary
word = id2word[[0][:1][0]]
#print (word)

[(0, 2), (1, 2), (2, 2), (3, 2), (4, 14), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 2), (15, 8), (16, 1), (17, 1), (18, 1), (19, 1)]


In [31]:
# Find the optimal number of Topics for LDA

def compute_coherence_values(dictionary, corpus,texts, limit, start,step,alpha, beta):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=alpha,
                                           per_word_topics=True,
                                           eta = beta)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [32]:
# Can take a long time to run.

#alpha_list = ['symmetric',0.3,0.5,0.7]
#beta_list = ['auto',0.3,0.5,0.7]


#for alpha in alpha_list:
        #for beta in beta_list:
            #model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=tok, start=2, limit=40, step=6,alpha=alpha,beta =beta)
            #print(f"alpha : {alpha} ; beta : {beta} ; Score : {coherence_values}")
            


In [33]:
# # Show graph
# limit=40; start=2; step=6;
# x = range(start, limit, step)
# plt.plot(x, coherence_values)
# plt.xlabel("Num Topics")
# plt.ylabel("Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

In [32]:
# compute the ladmodel with the best parameter after the tunning
# alpha string
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=32,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='symmetric',
                                           per_word_topics=True,
                                           eta = 0.7)

In [33]:
# Compute Coherence Score
coherencemodel = CoherenceModel(model=lda_model, texts=tok, dictionary=id2word, coherence='c_v')
coherence= coherencemodel.get_coherence()
print('\nCoherence Score: ', coherence)


Coherence Score:  0.38900823601447


In [167]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.000*"use" + 0.000*"model" + 0.000*"word" + 0.000*"data" + 0.000*"network" + 0.000*"language" + 0.000*"train" + 0.000*"learn" + 0.000*"sequence" + 0.000*"neural"


Topic: 1 
Words: 0.018*"word" + 0.016*"use" + 0.013*"model" + 0.011*"vector" + 0.010*"image" + 0.009*"sentence" + 0.008*"learn" + 0.007*"train" + 0.006*"set" + 0.006*"question"


Topic: 2 
Words: 0.000*"model" + 0.000*"use" + 0.000*"word" + 0.000*"language" + 0.000*"set" + 0.000*"learn" + 0.000*"system" + 0.000*"data" + 0.000*"result" + 0.000*"network"


Topic: 3 
Words: 0.022*"model" + 0.017*"word" + 0.016*"neural" + 0.015*"use" + 0.014*"network" + 0.011*"translation" + 0.009*"language" + 0.009*"input" + 0.009*"train" + 0.008*"machine"


Topic: 4 
Words: 0.000*"use" + 0.000*"model" + 0.000*"word" + 0.000*"data" + 0.000*"test" + 0.000*"method" + 0.000*"learn" + 0.000*"set" + 0.000*"system" + 0.000*"base"


Topic: 5 
Words: 0.017*"arc" + 0.014*"km" + 0.013*"flow" + 0.009*"hub" + 0.008*"route" + 0.006*"cost" 

# Finding the dominant topic in each sentence

In [34]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=tok):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
            # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
                if j == 0:  # => dominant topic
                    wp = ldamodel.show_topic(topic_num)
                    topic_keywords = ", ".join([word for word, prop in wp])
                    sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
                else:
                    break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=tok)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,3.0,0.9999,"model, word, neural, use, network, translation...","[statistical, machine, translation, draft, cha..."
1,1,16.0,0.9431,"word, model, use, representation, language, tr...","[find, learn, robust, joint, representation, c..."
2,2,5.0,0.9170,"arc, km, flow, hub, route, cost, algorithm, tr...","[available, online, system, engineering, route..."
3,3,11.0,0.8117,"term, weight, scheme, category, base, classifi...","[xx, xxx, xxx, inverse, category, frequency, b..."
4,4,30.0,1.0000,"use, model, test, method, word, document, like...","[university, dun, finding, structure, genome, ..."
5,5,16.0,0.9367,"word, model, use, representation, language, tr...","[beyond, stem, ultra, stem, improve, automatic..."
6,6,3.0,0.6615,"model, word, neural, use, network, translation...","[detect, phrase, mathematical, text, corpus, c..."
7,7,1.0,0.4418,"word, use, model, vector, image, sentence, lea...","[fast, accurate, sentiment, classification, us..."
8,8,26.0,0.9998,"measure, class, set, true, base, predict, pair...","[evaluation, measure, hierarchical, view, nove..."
9,9,3.0,0.9918,"model, word, neural, use, network, translation...","[generate, sequence, recurrent, neural, networ..."


In [37]:
len(df_dominant_topic)

50

In [38]:
df_soreted = df_dominant_topic.sort_values(by='Dominant_Topic', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
df_soreted

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
24,24,1.0,0.6266,"word, use, model, vector, image, sentence, lea...","[show, neural, image, caption, generator, goog..."
28,28,1.0,0.5927,"word, use, model, vector, image, sentence, lea...","[accepted, workshop, contribution, university,..."
18,18,1.0,0.8466,"word, use, model, vector, image, sentence, lea...","[neural, network, sentence, kim, new, york, un..."
29,29,1.0,0.8144,"word, use, model, vector, image, sentence, lea...","[accepted, workshop, contribution, metric, lea..."
15,15,1.0,0.8908,"word, use, model, vector, image, sentence, lea...","[distribute, representation, sentence, documen..."
14,14,1.0,0.9997,"word, use, model, vector, image, sentence, lea...","[open, question, answer, weakly, supervise, em..."
31,31,1.0,0.7556,"word, use, model, vector, image, sentence, lea...","[text, understanding, scratch, c, c, computer,..."
32,32,1.0,0.6531,"word, use, model, vector, image, sentence, lea...","[improve, semantic, representation, tree, stru..."
25,25,1.0,0.8915,"word, use, model, vector, image, sentence, lea...","[caption, visual, concept, back, fang, gupta, ..."
33,33,1.0,0.9996,"word, use, model, vector, image, sentence, lea...","[fast, accurate, dependency, parser, mohammad,..."


In [39]:
topics = df_soreted[['Dominant_Topic','Text']].values.tolist() 

In [40]:
topics_numbers = [item[0] for item in topics]

In [41]:
grouped_df = df_soreted.groupby('Dominant_Topic').agg(list)

In [70]:
grouped_df.head()

,Document_No,Topic_Perc_Contrib,Keywords,Text
Dominant_Topic,,,,
1.0,"[24, 28, 18, 29, 15, 14, 31, 32, 25, 33, 40, 7...","[0.6266000270843506, 0.5927000045776367, 0.846...","[word, use, model, vector, image, sentence, le...","[[show, neural, image, caption, generator, goo..."
3.0,"[38, 41, 42, 44, 30, 45, 46, 27, 39, 23, 0, 21...","[0.9991999864578247, 0.9997000098228455, 0.994...","[model, word, neural, use, network, translatio...","[[critical, review, recurrent, neural, network..."
5.0,[2],[0.9169999957084656],"[arc, km, flow, hub, route, cost, algorithm, t...","[[available, online, system, engineering, rout..."
9.0,[22],[0.6546000242233276],"[naive, bayes, class, probability, text, featu...","[[naive, bayes, text, introduction, theory, se..."
10.0,[37],[0.6431000232696533],"[domain, adaptation, target, source, data, lea...","[[journal, machine, learn, research, submit, p..."


In [72]:
index = grouped_df.index
grouped_df['Topic_num'] = index

In [73]:
grouped_df['Topic_num']

Dominant_Topic
1.0      1.0
3.0      3.0
5.0      5.0
9.0      9.0
10.0    10.0
11.0    11.0
15.0    15.0
16.0    16.0
17.0    17.0
24.0    24.0
26.0    26.0
30.0    30.0
Name: Topic_num, dtype: float64

In [74]:
df_char = grouped_df[['Topic_num', 'Text']]
df_char.head()

,Topic_num,Text
Dominant_Topic,,
1.0,1.0,"[[show, neural, image, caption, generator, goo..."
3.0,3.0,"[[critical, review, recurrent, neural, network..."
5.0,5.0,"[[available, online, system, engineering, rout..."
9.0,9.0,"[[naive, bayes, text, introduction, theory, se..."
10.0,10.0,"[[journal, machine, learn, research, submit, p..."


In [75]:
df_char.index.names = ['index']

In [76]:
df_char

,Topic_num,Text
index,,
1.0,1.0,"[[show, neural, image, caption, generator, goo..."
3.0,3.0,"[[critical, review, recurrent, neural, network..."
5.0,5.0,"[[available, online, system, engineering, rout..."
9.0,9.0,"[[naive, bayes, text, introduction, theory, se..."
10.0,10.0,"[[journal, machine, learn, research, submit, p..."
11.0,11.0,"[[xx, xxx, xxx, inverse, category, frequency, ..."
15.0,15.0,"[[effective, use, word, order, text, categoriz..."
16.0,16.0,"[[kim, character, aware, neural, language, mod..."
17.0,17.0,"[[end, end, memory, network, dept, computer, s..."


In [103]:
art = df_char['Text'].values

In [133]:
from itertools import chain
art_string = []

for a in art:
    corpus = ''
    for string in a[0]:
        corpus += " " + string  
    a = corpus
    art_string.append(corpus)

In [136]:
art_string[2]

' available online system engineering route optimization algorithm solution web service engineering wuhan university abstract many modern service system rely network hub facility help concentrate flow freight passenger exploit economy scale transportation whereas possible defect bypass cost cause hub seem paper employ novel optimal hub speak network base decision approach unite bypass cost congestion effect algorithm time effect compare set computational work perform personal computer data postal operation draw conclusion approach present paper much good time traditional way keywords route engineering introduction service orient foresees creation business application independently develop similar compete service correspond functional description best set web select run time order maximize quality minimize price end user application workload vary order magnitude even within business optimization perform process execution start iterate run time order take account workload real world pric

In [137]:
df_char['corpus'] = art_string

/tmp/ipykernel_9333/423988610.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_char['corpus'] = art_string


In [139]:
df_char.head()

,Topic_num,Text,test,corpus
index,,,,
1.0,1.0,"[[show, neural, image, caption, generator, goo...","[['show', 'neural', 'image', 'caption', 'gener...",show neural image caption generator google al...
3.0,3.0,"[[critical, review, recurrent, neural, network...","[['critical', 'review', 'recurrent', 'neural',...",critical review recurrent neural network sequ...
5.0,5.0,"[[available, online, system, engineering, rout...","[['available', 'online', 'system', 'engineerin...",available online system engineering route opt...
9.0,9.0,"[[naive, bayes, text, introduction, theory, se...","[['naive', 'bayes', 'text', 'introduction', 't...",naive bayes text introduction theory sebastia...
10.0,10.0,"[[journal, machine, learn, research, submit, p...","[['journal', 'machine', 'learn', 'research', '...",journal machine learn research submit publish...


In [143]:
data = df_char[['Topic_num', 'corpus']].to_numpy()

In [189]:
input_features = []
output = []
total_n_v = 0

for article in data[0:1]:
    raw_text = article[1]
    print("Topic Number", article[0])
    chars = sorted(list(set(raw_text)))
    char_to_int = dict((c, i) for i, c in enumerate(chars))
    int_to_char = dict((i, c) for i, c in enumerate(chars))

    print(chars)
    print("Char to int" , char_to_int)
    print("Int to char", int_to_char)
    
    # Prints the total characters and character vocab size
    n_chars = len(raw_text)
    n_vocab = len(chars)
    
    total_n_v += n_vocab

    print("The number of total characters are", n_chars)
    print("\nThe character vocab size is", n_vocab)
    
    #Prepares dataset where the input is sequence of 100 characters and target is next character.
    seq_length = 100

    dataX = []
    dataY = []

    for i in range(0, n_chars - seq_length, 1):
          seq_in = raw_text[i:i + seq_length]
          seq_out = raw_text[i + seq_length]
          dataX.append([char_to_int[char] for char in seq_in])
          dataY.append(char_to_int[seq_out])

    n_patterns = len(dataX)
    print ("Total Patterns: ", n_patterns)
    
    from keras.utils import np_utils
    # reshapes X to be [samples, time steps, features]
    X = np.reshape(dataX, (n_patterns, seq_length, 1))
    print("The X input", len(X))
    input_features.append(X)
    # one hot encodes the output variable
    Y = np_utils.to_categorical(dataY)
    print("The Y output", Y[1])
    print("The Y output", Y[0])
    output.append(Y)   

Topic Number 1.0
[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Char to int {' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
Int to char {0: ' ', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}
The number of total characters are 19427

The character vocab size is 27
Total Patterns:  19327
The X input 19327
The Y output [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
The Y output [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 0.]


In [190]:
input_features[0].shape

(19327, 100, 1)

In [197]:
real_Y = output[0]

In [173]:
embedding_dim =100
max_length =100

In [174]:
from keras.layers import Dropout

In [195]:
model1 = Sequential()
model1.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam',  metrics=['accuracy'])
model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 100)          2700      
                                                                 
 lstm_4 (LSTM)               (None, 100, 256)          365568    
                                                                 
 dropout_4 (Dropout)         (None, 100, 256)          0         
                                                                 
 lstm_5 (LSTM)               (None, 256)               525312    
                                                                 
 dropout_5 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 27)                6939      
                                                                 
Total params: 900,519
Trainable params: 900,519
Non-tr

In [198]:
model1.fit(input_features, real_Y, epochs = 20, batch_size=128)

Epoch 1/20
151/151 [==============================] - 119s 774ms/step - loss: 2.8626 - accuracy: 0.1643
Epoch 2/20
151/151 [==============================] - 113s 750ms/step - loss: 2.5495 - accuracy: 0.2519
Epoch 3/20
151/151 [==============================] - 112s 740ms/step - loss: 2.1726 - accuracy: 0.3551
Epoch 4/20
151/151 [==============================] - 109s 720ms/step - loss: 1.9058 - accuracy: 0.4327
Epoch 5/20
151/151 [==============================] - 107s 708ms/step - loss: 1.6758 - accuracy: 0.5042
Epoch 6/20
151/151 [==============================] - 107s 709ms/step - loss: 1.4970 - accuracy: 0.5518
Epoch 7/20
151/151 [==============================] - 106s 702ms/step - loss: 1.3668 - accuracy: 0.5910
Epoch 8/20
151/151 [==============================] - 104s 692ms/step - loss: 1.2632 - accuracy: 0.6165
Epoch 9/20
151/151 [==============================] - 106s 699ms/step - loss: 1.1708 - accuracy: 0.6428
Epoch 10/20
151/151 [==============================] - 107s 710m

In [201]:
# Generates the sequence similar to above methods. Gets the generated string using the model.
def predict_next_n_chars(pattern, n):
    for i in range(n):
      x = np.reshape(pattern, (1, len(pattern), 1))
      prediction = model1.predict(x, verbose=0)
      print (int_to_char[np.argmax(prediction)], end = '')   #get next char index.
      seq_in = [int_to_char[value] for value in pattern]
      pattern.append(np.argmax(prediction))
      pattern = pattern[1:len(pattern)]

In [202]:
#picks a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])
print ("Seed -",  input_str, sep = '\n\n')
print ("\nGenerated string -\n")

predict_next_n_chars(pattern, 200)

Seed -

es image adequately main inspiration work come recent ad machine task transform sentence write sourc

Generated string -

e target maximize likelihood sentence give image description give image see approach image annotation use neural network encodes variable length input decoder generates complete sentence generate desc

In [203]:
input_str = "Many modern service systems rely on a network of hub facilities to help concentrate flows of freight or passengers"

#Uses the first 100 characters from given input_str as input to generate next 200 characters. 
input_str = input_str.lower()
input_string = ''
for each in input_str:
    if each in chars:
           if (len (input_string) < 100):
                input_string += each

pattern = []
pattern.append([char_to_int[char] for char in input_string])

print ("Seed -",  input_str, sep = '\n\n')
print ("\nGenerated string -\n")
predict_next_n_chars(pattern[0], 200)

Seed -

many modern service systems rely on a network of hub facilities to help concentrate flows of freight or passengers

Generated string -

 memory block contains cell control layer apply thus noise convert train maximize likelihood sentence give image description give image see approach image annotation use neural network encodes variabl

In [205]:
input_str = "recently most of the research based on the Neural networks models, because they are fast and accurate models "

#Uses the first 100 characters from given input_str as input to generate next 200 characters. 
input_str = input_str.lower()
input_string = ''
for each in input_str:
    if each in chars:
           if (len (input_string) < 100):
                input_string += each

pattern = []
pattern.append([char_to_int[char] for char in input_string])

print ("Seed -",  input_str, sep = '\n\n')
print ("\nGenerated string -\n")
predict_next_n_chars(pattern[0], 200)

Seed -

recently most of the research based on the neural networks models, because they are fast and accurate models 

Generated string -

 deep convolution neural network encodes variable length input decoder generates complete sentence generate description give image see approach image annotation use neural network encodes variable len